# ***OpenEO - [GIS4 wildlife movement analytics](https://www.gis4-wildlife.com/)***

## ***1. Fetch data from openEO***

In [2]:
import openeo
import pandas as pd
import geopandas as gpd
import xarray as xr
HDF5_USE_FILE_LOCKING=False
import matplotlib.pyplot as plt

In [ ]:
# Documentation: https://docs.openeo.cloud/getting-started/python/#connect-to-openeo-platform-and-explore

# API connection to openEO
connection = openeo.connect("openeo.cloud").authenticate_oidc()

In [ ]:
# connection.list_collections()

In [ ]:
# connection.list_processes()

In [ ]:
# Fetch data from boundary, dates, and specific band with SST
datacube = connection.load_collection(
  "SENTINEL3_SLSTR",
  spatial_extent={"west":-100.00,"south":-8.00,"east":-76.00,"north":8.00},
  temporal_extent=["2022-02-01", "2022-02-10"],
  bands=["S9"]
)

In [ ]:
# Willing to create a datacube with weekly average
datacube = datacube.mean_time()

In [ ]:
# Download file in local disk
datacube.download("sst.nc", format="netcdf")

## ***2. Read and transform downloaded data***

In [3]:
# Read the NetCDF file and transformed in DataFrame 
DS = xr.open_dataset('results/sst.nc')

DF = DS.to_dataframe()

In [4]:
# index removed
DF = DF.reset_index(drop=False)

# from Kelvin to Celsius
DF['S9'] = DF['S9'] - 273.15 

In [5]:
DF.head()

,x,y,crs,S9
0,-99.99504,7.995040,b'',-31.026678
1,-99.99504,7.985119,b'',-33.579993
2,-99.99504,7.975198,b'',-34.620001
3,-99.99504,7.965278,b'',-35.069998
4,-99.99504,7.955357,b'',-32.023337


In [ ]:
DF['S9'].min()

In [ ]:
DF['S9'].max()

In [6]:
# removal of outliers. Assumed that there is non negative values in the equatorial region
DF = DF.loc[DF['S9']>0]

In [ ]:
DF['S9'].min()

In [ ]:
DF['S9'].describe()

In [ ]:
DF.to_csv('sst_openEO.csv')

## ***3. Visualization***

![sst](results/sst_gps_s9.png)

In [ ]:
DS.S9.plot(x='x', y='y', figsize=(18,12), cmap='Spectral'); #, vmin=0, vmax=5

plt.title('Exploration openEO - SST - Sentinel3');

plt.savefig('sst_gps_S9.png', dpi=300)